## Graph Search, Shortest Paths, and Data Structures
### Programming Assignment 1
The file contains the edges of a directed graph. Vertices are labeled as positive integers from 1 to 875714. Every row indicates an edge, the vertex label in first column is the tail and the vertex label in second column is the head (recall the graph is directed, and the edges are directed from the first column vertex to the second column vertex). So for example, the 11^{th}11 
th
  row looks liks : "2 47646". This just means that the vertex with label 2 has an outgoing edge to the vertex with label 47646

Your task is to code up the algorithm from the video lectures for computing strongly connected components (SCCs), and to run this algorithm on the given graph.

Output Format: You should output the sizes of the 5 largest SCCs in the given graph, in decreasing order of sizes, separated by commas (avoid any spaces). So if your algorithm computes the sizes of the five largest SCCs to be 500, 400, 300, 200 and 100, then your answer should be "500,400,300,200,100" (without the quotes). If your algorithm finds less than 5 SCCs, then write 0 for the remaining terms. Thus, if your algorithm computes only 3 SCCs whose sizes are 400, 300, and 100, then your answer should be "400,300,100,0,0" (without the quotes). (Note also that your answer should not have any spaces in it.)

WARNING: This is the most challenging programming assignment of the course. Because of the size of the graph you may have to manage memory carefully. The best way to do this depends on your programming language and environment, and we strongly suggest that you exchange tips for doing this on the discussion forums.

In [1]:
from collections import defaultdict

class Graph:
    def __init__(self, vertices):
        self.V= vertices #No. of vertices 
        self.graph = defaultdict(list)

    def addEdge(self, u,v):
        self.graph[u].append(v)
    
    def reverse_graph(self):
        reversed = Graph(self.V)
        for i in self.graph:
            for j in self.graph[i]:
                reversed.addEdge(j,i)
        return reversed

    def BFS(self, s):
        visited = [False]*(self.V)
        queue = []
        queue.append(s)
        visited[s]= True

        while queue:
            s = queue.pop(0)
            print(str(s), end='')

            for i in self.graph[s]:
                if visited[i] == False:
                    visited[i]= True
                    queue.append(i)

    def DFS_loop(self,v, visited, scc):
        visited[v] = True
        #print(v, end='')
        for i in self.graph[v]:
            if visited[i] == False:
                self.DFS_loop(i, visited)
        scc+=1

    def DFS(self, v, stack):
        visited= [False]*(self.V)
        self.DFS_loop(v, visited)

    def fillOrder(self,v,visited, stack): 
            # Mark the current node as visited 
            visited[v]= True
            #Recur for all the vertices adjacent to this vertex 
            for i in self.graph[v]: 
                if visited[i]==False: 
                    self.fillOrder(i, visited, stack) 
            stack = stack.append(v) 
    
    def printSCC(self):
        scc_list = []
        visited = [False]*self.V
        stack= []
        for i in range(self.V):
            if visited[i]==False:
                self.fillOrder(i,visited, stack)
        #print('stack:', stack)
        g_rev = g.reverse_graph()
        visited = [False]*self.V
        scc = 0
        while stack:
            i = stack.pop()
            if visited[i]==False:
                g_rev.fillOrder(i, visited, scc)
                #print('')
                scc_list.append(scc)
                scc= 0
        return scc_list


In [ ]:
# WARNING: This will likely to end in stack overflow
source=[]
f = lambda x,y : [int(x),int(y)]
for line in open('SCC.txt'):
    source.append(f(line.split()[0], line.split()[1]))
N = 875714+1
g = Graph(N)
for  i in range(1,len(source)):
    u,v = source[i]
    g.addEdge(u,v)
import sys
sys.setrecursionlimit(N)
scc_list = g.printSCC()

In [1]:
class SccFinder(object):
    def __init__(self, input_file):
        self.scc_list = []
        with open(input_file) as file:
            self.finish_order = []
            self._graph = {}
            for line in file:
                (from_v, to_v) = tuple(number for number in line.split())
                self._add_edge_to_graph(int(from_v), int(to_v))

    def _add_edge_to_graph(self, from_v, to_v):
        if from_v in self._graph:
            self._graph[from_v].append(to_v)
        else:
            self._graph[from_v] = [to_v]
        if to_v in self._graph:
            self._graph[to_v].append(-from_v)
        else:
            self._graph[to_v] = [-from_v]

    def compute_finish_times(self):
        visited_nodes, finished_nodes = set(), set()
        for vertex in self._graph.keys():
            if vertex in visited_nodes:
                continue
            nodes_stack = [vertex]
            while nodes_stack:
                node = nodes_stack.pop()
                if node not in visited_nodes:
                    visited_nodes.add(node)
                    nodes_stack.append(node)
                    neighbors = (-edge for edge in self._graph[node] if edge < 0)
                    for neighbor in neighbors:
                        if neighbor not in visited_nodes:
                            nodes_stack.append(neighbor)
                else:
                    if node not in finished_nodes:
                        self.finish_order.append(node)
                        finished_nodes.add(node)

    def compute_sccs(self):
        visited_nodes = set()
        assert (len(self.finish_order) == len(self._graph))
        for i in reversed(self.finish_order):
            if i in visited_nodes:
                continue
            nodes_stack = [i]
            size = 0
            while nodes_stack:
                node = nodes_stack.pop()
                if node not in visited_nodes:
                    size += 1
                    visited_nodes.add(node)
                    nodes_stack.append(node)
                    neighbors = (edge for edge in self._graph[node] if edge > 0)
                    for neighbor in neighbors:
                        if neighbor not in visited_nodes:
                            nodes_stack.append(neighbor)
            self.scc_list.append(size)
        self.scc_list.sort(reverse=True)
        #print(self.scc_list[:5])

if __name__ == "__main__":
    scc_finder = SccFinder("SCC.txt")
    scc_finder.compute_finish_times()
    scc_finder.compute_sccs()
    #expected_sccs = [434821, 968, 459, 313, 211]
    print(scc_finder.scc_list[:5])

#Output: [434821, 968, 459, 313, 211]

[434821, 968, 459, 313, 211]
[434821, 968, 459, 313, 211]
